In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "http://www.moscow_city.vybory.izbirkom.ru/region/moscow_city"
response = requests.get(url)
page = BeautifulSoup(response.content)

In [ ]:
votes = {}

admin_data = pd.read_html(url)[7]
admin_data.columns = ['admin','municipal']
admin_data = df.iloc[1:,:]
admin_data = df.fillna(method='ffill')
admin_data['municipal'] = df['municipal'].str.replace('Выборы депутатов Совета депутатов муниципального округа ', '')
admin_data['municipal'] = df['municipal'].str.replace(' в городе Москве', '')
admin_data['admin'] = df['admin'].str.replace('город Москва ','')
admin_data['admin'] = df['admin'].str.replace('административный округ','АО')
votes['admin'] = admin_data

main_links = page.find_all('a')
for main_tag in main_links:
    main_link = main_tag.get('href',None)
    if 'region=77'in main_link:
        name = main_tag.text.replace('Выборы депутатов Совета депутатов муниципального округа ', '').replace(' в городе Москве', '')
        votes[name] = {'votes':{}}
        region_page = BeautifulSoup(requests.get(main_link).content)
        tik_tags = region_page.find_all('option')
        region_tags = region_page.find_all('a')
        for region_tag in region_tags:
            region_link = region_tag.get('href',None)
            if 'type=220'in region_link:
                candidates = []
                for i in range(1,6):
                    candidates_link = region_link + '&number=' + str(i)
                    try:
                        candidates_df = pd.read_html(candidates_link, encoding='cp1251')[5].iloc[:,1:]
                        candidates_df.columns = ['name','date', 'party', 'okrug', 'drived', 'registered', 'elected']
                        candidates.append(candidates_df)
                    except:
                        pass
                candidates = pd.concat(candidates, axis=1)
                votes[name]['candidates'] = candidates
        for tik_tag in tik_tags:
            tik_link = tik_tag.get('value',None)
            if tik_link is not None:
                okrug = int(tik_tag.text[-1])
                tik_page = BeautifulSoup(requests.get(tik_link).content)
                okrug_tags = tik_page.find_all('a')
                for okrug_tag in okrug_tags:
                    okrug_link = okrug_tag.get('href',None)
                    if 'type=424'in okrug_link:
                        dd = pd.read_html(okrug_link)
                        okrug_df = pd.concat([dd[6], dd[7]], axis=1)
                        okrug_columns = ['parameter', 'overall'] + list(okrug_df.iloc[0,:])[3:]
                        candidates_columns = ['name', 'overall'] + list(okrug_df.iloc[0,:])[3:]
                        okrug_stats = okrug_df.iloc[1:13,1:]
                        candidates_stats = okrug_df.iloc[14:,1:]
                        okrug_stats.columns = okrug_columns
                        candidates_stats.columns = candidates_columns
                        okrug_stats = {'okrug_stats': okrug_stats, 'candidates_stats': candidates_stats}
                        votes[name]['votes'][okrug] = okrug_stats